# Job Position Statistic

For an information technology student, finding out the positions in the IT market is a very necessary thing. In this notebook, I will summarize the number of positions currently recruiting and compare that number of positions and based on that statisitc, we can understand the recruitment needs of companies for each specific position.



In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


class Job(webdriver.Firefox):
    def __init__(self, name_job, url= 'https://www.linkedin.com/home',list_jobs = [], job_infor = {}, ):
        self.url = url;
        self.name_job = name_job
        self.list_jobs = list_jobs
        self.job_infor = job_infor
        super(Job, self).__init__()

    def get_list_jobs(self):
        return self.list_jobs
    
    def set_list_job(self, name_job):
        self.list_jobs = list_jobs
    
    def goto_page(self):
        self.get(self.url)
        

    def search(self):
        self.find_element(By.LINK_TEXT, "Jobs").click()
        time.sleep(4)
        field = self.find_element(By.NAME, "keywords")
        location = self.find_element(By.NAME, "location")
        
        time.sleep(3)
        field.send_keys(self.name_job)
            
        time.sleep(2)
        location.send_keys(Keys.CONTROL + "a")
        location.send_keys(Keys.DELETE)
        location.send_keys("Vietnam")

        time.sleep(2)

        location.send_keys(Keys.ENTER)
     
    
    def extract_data(self):
        list_to_extract_jobs = self.find_element(By.CLASS_NAME, "jobs-search__results-list")
        
        list_li = list_to_extract_jobs.find_elements(By.TAG_NAME, "li")

        for item in list_li:
            jobs_title = item.find_element(By.TAG_NAME, "h3").text
            jobs_company = item.find_element(By.TAG_NAME, "h4").text
            jobs_location = item.find_element(By.CLASS_NAME, "job-search-card__location").text
            self.job_infor = {'Position': jobs_title, 'Company': jobs_company, 'Location': jobs_location}
            self.list_jobs.append(self.job_infor)
        
        
    def load_page(self):
        self.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
    def second_load(self):
        for i in range(20):
            main = self.find_element(By.ID, "main-content")
            div = main.find_element(By.CLASS_NAME, "two-pane-serp-page__results-list")
            try:
                button = div.find_element(By.TAG_NAME, "button").click()
            except:
                pass
            self.load_page()
    
    def add_to_file(self):
        df = pd.DataFrame(self.list_jobs)
        match self.name_job:
            case "Software":
                return df.to_csv("Software_position.csv", mode="w", index=False, header=True)
            case "Hardware":
                return df.to_csv("Hardware_position.csv", mode="w", index=False, header=True)
            case "Data":
                return df.to_csv("Data_position.csv", mode="w", index=False, header=True)
            case "Security":
                return df.to_csv("Security_position.csv", mode="w", index=False, header=True)
        

In this case, I will give several keywords to review all field in Linkedin such as: **Sofware**, **Hardware**, **Data** and **Security**. The reason why I use several keywords instead of just one like **IT** is that when i use **IT** keyword, data becomes cluttered, difficult to sort by field as well as more difficult to sort by level.

In [4]:
# Step 1: Inspect Data Source 
list_keywords = ["Software", "Hardware", "Data", "Security"]

for i in list_keywords:
    job = Job(i)
    job.goto_page()

    job.search()
    time.sleep(4)
    
# Step 2: Scrape HTML content & Step 3: parse HTML Code    
    for i in range(6):
        job.load_page()
    
    job.second_load()
    job.extract_data()

# Step 4: Export into csv file
    job.add_to_file()
    job.get_list_jobs().clear()

NoSuchElementException: Message: Unable to locate element: [name="keywords"]
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.jsm:12:1
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.jsm:404:5
element.find/</<@chrome://remote/content/marionette/element.js:291:16
